S&P500 stock data:
http://pages.swcp.com/stocks/


n = pd.read_csv('http://chart.finance.yahoo.com/table.csv?s=ACIU&a=7&b=9&c=1996&d=10&e=19&f=2016&g=d&ignore=.csv')

http://www.eoddata.com/download.aspx

In [386]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData
from edgerdb import helper_functions as hlp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


meta = MetaData()

engine = create_engine('postgresql://analyst:@localhost:5432/edgar')

messages = Table('filings', meta, autoload=True, autoload_with=engine)

def sql_to_df(query, connection):
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [387]:
%matplotlib inline

In [2]:
cik_and_tickers = sql_to_df('select * from temp_cik_to_ticker;', engine.connect()).set_index(keys='index')

In [3]:
cik_and_tickers.head()

,cik,company_name,exchange,isactive,symbol
index,,,,,
0,1591890,"1347 PROPERTY INSURANCE HOLDINGS, INC.",NASDAQ,1,PIH
1,1508475,"21VIANET GROUP, INC.",NASDAQ,1,VNET
2,1459417,"2U, INC.",NASDAQ,1,TWOU
3,1295484,"51JOB, INC.",NASDAQ,1,JOBS
4,1295484,"51JOB, INC.",NASDAQ,1,JOBS


In [4]:
tickers = cik_and_tickers['symbol'].tolist()


In [57]:
import requests
from yahoo_finance import Share
from pprint import pprint

In [58]:
yahoo = Share('YHOO')
print(yahoo.get_open())
print(yahoo.get_price())
pprint(yahoo.get_historical('2014-04-25', '2014-04-29'))

None
41.45
[{'Adj_Close': '35.830002',
  'Close': '35.830002',
  'Date': '2014-04-29',
  'High': '35.889999',
  'Low': '34.119999',
  'Open': '34.369999',
  'Symbol': 'YHOO',
  'Volume': '28736000'},
 {'Adj_Close': '33.990002',
  'Close': '33.990002',
  'Date': '2014-04-28',
  'High': '35.00',
  'Low': '33.650002',
  'Open': '34.669998',
  'Symbol': 'YHOO',
  'Volume': '31019200'},
 {'Adj_Close': '34.48',
  'Close': '34.48',
  'Date': '2014-04-25',
  'High': '35.099998',
  'Low': '34.290001',
  'Open': '35.029999',
  'Symbol': 'YHOO',
  'Volume': '19401600'}]


In [97]:
# Columns will need to be added to top of file after download
sp500 = pd.read_csv('data/sp500hst.txt')

In [99]:
len(sp500['symbol'].unique())

524

In [191]:
tickers[0:10]

['PIH',
 'VNET',
 'TWOU',
 'JOBS',
 'JOBS',
 'CAFD',
 'ABEO',
 'ABEOW',
 'ABIL',
 'ACIU']

In [268]:
r2016 = pd.read_csv('data/5_YEAR_NASDAQ/NASDAQ_20160101.csv')

In [270]:
print(len(r2016['Symbol'].tolist()))
print(len(set(r2016['Symbol'].tolist() + tickers)))

2812
13534


In [288]:
def in_both(first_li, second_li):
    both = []
    for x in second_li:
        if x in first_li:
            both.append(x)
    return both


def find_csv_files_in_dir(directory):
    paths = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".csv"):
                paths.append(filepath)
    return paths


def concat_eod_data(list_of_paths):
    '''
        This functions takes as input a list of paths to csv files containg eod stock data and join the files
        into a single dataframe.
    '''
    df = pd.DataFrame(columns=['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    for path in list_of_paths:
        temp_df = pd.read_csv(path)
        df = pd.concat([df, temp_df])
    return df

In [507]:
months = {'Jan': '01', 
          'Feb': '02', 
          'Mar': '03', 
          'Apr': '04', 
          'May': '05', 
          'Jun': '06',
          'Jul': '07',
          'Aug': '08', 
          'Sep': '09', 
          'Oct': '10',
          'Nov': '11', 
          'Dec': '12'}

five_year_nasdaq_files = find_csv_files_in_dir('data/5_YEAR_NASDAQ')
five_year_nasdaq_df = concat_eod_data(five_year_nasdaq_files)
five_year_nasdaq_df['Date'] = five_year_nasdaq_df['Date'].apply(lambda x: x.replace(x[3:6], months[x[3:6]]))
five_year_nasdaq_df['Date'] = five_year_nasdaq_df['Date'].apply(lambda x: x.split('-')[2] + x.split('-')[1] + x.split('-')[0])


In [546]:

five_year_nasdaq_df.head()

,Symbol,Date,Open,High,Low,Close,Volume
0,AAME,20110103,2.0600,2.0600,2.0600,2.0600,1400
1,AAON,20110103,8.4622,8.6163,8.4503,8.5955,103000
2,AAPL,20110103,46.5200,47.1800,46.4057,47.0814,15897200
3,AAWW,20110103,56.1200,57.0900,55.7900,55.8600,250400
4,AAXJ,20110103,64.6000,64.8200,64.5000,64.5800,181600


In [ ]:
tickers_in_df = five_year_nasdaq_df['Symbol'].unique().tolist()
tickers_in_both = in_both(tickers_in_df, tickers)
tickers_cik_in_both = cik_and_tickers[cik_and_tickers['symbol'].isin(tickers_in_both)]

In [356]:
filings = sql_to_df('select * from filings where cik in {};'.format(tuple(tickers_cik_in_both['cik'])), engine.connect())

In [358]:
len(filings)

1297972

In [ ]:
tickers_cik = pd.DataFrame(tickers_cik_in_both, columns=['cik', 'symbol'])
symbol_to_filings = pd.merge(left=filings, right=tickers_cik[tickers_cik['cik'].isin(
            tickers_cik['cik'].unique())], left_on='cik', right_on='cik')
symbol_to_filings = pd.DataFrame(symbol_to_filings, columns=['cik', 'symbol', 'company_name', 'type', 'date', 'path'])

In [389]:
symbol_to_filings.head()

,cik,symbol,company_name,type,date,path
0,1357450,HOLI,HLS SYSTEMS INTERNATIONAL LTD,20-F,20081212,edgar/data/1357450/0001204459-08-002410.txt
1,1357450,HOLI,HLS SYSTEMS INTERNATIONAL LTD,424B4,20070814,edgar/data/1357450/0001144204-07-042666.txt
2,1357450,HOLI,HLS SYSTEMS INTERNATIONAL LTD,425,20060403,edgar/data/1357450/0001144204-06-013505.txt
3,1357450,HOLI,HLS SYSTEMS INTERNATIONAL LTD,6-K,20080129,edgar/data/1357450/0001144204-08-004735.txt
4,1357450,HOLI,HLS SYSTEMS INTERNATIONAL LTD,6-K,20080220,edgar/data/1357450/0001144204-08-010616.txt


In [ ]:
def symbol_to_path(symbol, base_dir="data"):
    """Return CSV file path given ticker symbol."""
    return os.path.join(base_dir, "{}.csv".format(str(symbol)))


def get_data(symbols, dates):
    """Read stock data (adjusted close) for given symbols from CSV files."""
    df = pd.DataFrame(index=dates)
    if 'SPY' not in symbols:  # add SPY for reference, if absent
        symbols.insert(0, 'SPY')
    count = 0
    for symbol in symbols:
        # TODO: Read and join data for each symbol
        path = symbol_to_path(symbol)
        temp_df = pd.read_csv(path, index_col='Date', 
                             parse_dates=True, usecols=['Date', 'Adj Close'], na_values=['nan'])
        temp_df = temp_df.rename(columns={'Adj Close': symbol})
        df = df.join(temp_df)
        if symbol == 'SPY':
            df = df.dropna()
    return df


def plot_data(df, title='Stock Prices', ylabel="Price", xlabel="Date"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()
    
    
def test_run():
    # Read data
    dates = pd.date_range('2012-01-01', '2012-12-31')
    symbols = ['SPY']
    df = get_data(symbols, dates)
    
    # Plot SPY data, retain matplotlib axis object
    ax = df['SPY'].plot(title='SPY rolling mean', label='SPY')
    
    # Compute the rolling mean using a 20 day window
    rm_SPY = pd.rolling_mean(df['SPY'], window=20)
    
    # Add rolling mean to same plot
    rm_SPY.plot(label='Rolling mean', ax=ax)
    
    # Add axis labels and legend
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend(loc='upper left')
    plt.show()
    
    
    
if __name__ == "__main__":
    test_run()

In [544]:
t = pd.DataFrame(five_year_nasdaq_df[five_year_nasdaq_df['Symbol'] == 'AAPL'].sort_values(by=['Date'], ascending=True))
#t['Date'] = t['Date'].apply(lambda x: x.replace(x[3:6], months[x[3:6]]))
t['Date'] = t['Date'].apply(lambda x: x.split('-')[2] + x.split('-')[1] + x.split('-')[0])
#t['Date'] = t['Date'].apply(lambda x: x.replace('-', ''))
# csi_stock_data['company_name'] = csi_stock_data['company_name'].str.upper()


In [545]:
t

,Symbol,Date,Open,High,Low,Close,Volume
2,AAPL,20130101,76.0243,76.0243,76.0243,76.0243,0
4,AAPL,20140101,80.1457,80.1457,80.1457,80.1457,0
4,AAPL,20150101,110.3800,110.3800,110.3800,110.3800,0
6,AAPL,20160101,105.2600,105.2600,105.2600,105.2600,0
2,AAPL,20110201,48.7571,49.3786,48.7114,49.2900,15236800
2,AAPL,20120201,65.4871,65.5700,65.0786,65.1700,9644300
2,AAPL,20130201,65.5886,65.6400,64.0500,64.8029,19266700
6,AAPL,20160201,96.4700,96.7100,95.4000,96.4300,40943500
2,AAPL,20110301,50.7814,50.8171,49.6686,49.9014,16290300
2,AAPL,20120301,78.2857,78.3157,76.9671,77.7814,24401800
